# API testing

# Choosing which API

### Alpha Vantage API

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
def get_alpha_vantage_news(api_key, ticker, topic=None):
    """
    Fetches news articles about `ticker` from the Alpha Vantage News API.
    Optionally filters by `topic` (e.g., "technology", "finance", etc.).
    Returns a list of dicts with article info (title, summary, sentiment, etc.).
    """

    # Alpha Vantage 'NEWS_SENTIMENT' endpoint
    url = "https://www.alphavantage.co/query"

    # Base params
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "apikey": api_key
    }

    # If a topic was provided, include it in the params
    # NOTE: The Alpha Vantage parameter for filtering by topic is "topics".
    # E.g., topics=technology,ipo
    if topic:
        params["topics"] = topic

    response = requests.get(url, params=params)
    print("Request URL:", response.url)  # Debugging: see the final request
    data = response.json()

    # Check for errors
    if "feed" not in data:
        print(f"Error fetching news from Alpha Vantage: {data}")
        return []

    articles = data["feed"]
    return articles


def main():
    # 1. Prompt user for inputs
    ticker = input("Enter the company ticker symbol (e.g., AAPL, TSLA): ").upper()
    limit_str = input("How many news articles would you like to see? (default 5): ")
    
    print("\nAvailable topics include:")
    print("  blockchain, earnings, ipo, mergers_and_acquisitions, financial_markets,")
    print("  economy_fiscal, economy_monetary, economy_macro, energy_transportation,")
    print("  finance, life_sciences, manufacturing, real_estate, retail_wholesale, technology\n")
    
    topic_input = input("Enter a topic from the list above (or leave blank for none): ").strip()

    # Convert limit input to int
    try:
        limit = int(limit_str)
    except ValueError:
        # If user input is empty or invalid, default to 5
        limit = 5

    # 2. Your Alpha Vantage API key (replace with your actual key)
    ALPHA_VANTAGE_API_KEY = ""

    # 3. Retrieve news articles (potentially filtered by topic)
    articles = get_alpha_vantage_news(ALPHA_VANTAGE_API_KEY, ticker, topic=topic_input)

    # If Alpha Vantage does not respect a 'limit' parameter for news,
    # we apply the limit client-side:
    articles = articles[:limit]

    # 4. Print out the results
    if not articles:
        print("No articles found or there was an error.")
        return

    print(f"\n=== Top {limit} News Articles for {ticker} (Topic: {topic_input or 'None'}) ===")
    for i, article in enumerate(articles, start=1):
        title = article.get("title", "No Title")
        summary = article.get("summary", "No Summary")
        sentiment = article.get("overall_sentiment_label", "No Sentiment")
        url = article.get("url", "No URL")

        print(f"\nArticle {i}:")
        print(f"Title: {title}")
        print(f"Sentiment: {sentiment}")
        print(f"Summary: {summary[:200]}...")  # limit summary preview
        print(f"Link: {url}")

In [5]:
if __name__ == "__main__":
    main()


Available topics include:
  blockchain, earnings, ipo, mergers_and_acquisitions, financial_markets,
  economy_fiscal, economy_monetary, economy_macro, energy_transportation,
  finance, life_sciences, manufacturing, real_estate, retail_wholesale, technology

Request URL: https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=TSLA&apikey=RAL8AFRU7F2Y9ZPI

=== Top 5 News Articles for TSLA (Topic: None) ===

Article 1:
Title: AI's $19.9 Trillion Impact Could Create The First Trillionaire-But Mark Cuban Says It Won't Be Musk Or Bezos - Alphabet  ( NASDAQ:GOOGL ) , Amazon.com  ( NASDAQ:AMZN ) 
Sentiment: Somewhat-Bullish
Summary: As of this month, Elon Musk, CEO of Tesla TSLA and SpaceX, is still the world's richest person, with a net worth of around $393.2 billion, according to Forbes. While that's an enormous fortune, some e...
Link: https://www.benzinga.com/news/25/02/43903135/ais-19-9-trillion-impact-could-create-the-first-trillionaire-but-mark-cuban-says-it-wont-be-musk-or-b

## FinnHub API

In [ ]:
def get_finnhub_news(api_key, ticker):
    """
    Fetches news articles about `ticker` from the Finnhub 'company-news' endpoint.
    Returns a list of dicts with article info (headline, summary, etc.).
    """

    # Define a date range: e.g., 30 days in the past up to today
    to_date = datetime.now().strftime("%Y-%m-%d")
    from_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")

    url = "https://finnhub.io/api/v1/company-news"
    params = {
        "symbol": ticker,
        "from": from_date,
        "to": to_date,
        "token": api_key
    }

    response = requests.get(url, params=params)
    print("Request URL:", response.url)  # Debugging: see the final request
    data = response.json()

    # Finnhub returns a list of articles (or an error message)
    if not isinstance(data, list):
        print(f"Error fetching news from Finnhub: {data}")
        return []

    return data


def main():
    # 1. Prompt user for inputs
    ticker = input("Enter the company ticker symbol (e.g., AAPL, TSLA): ").upper()
    limit_str = input("How many news articles would you like to see? (default 5): ")

    # Convert limit input to int
    try:
        limit = int(limit_str)
    except ValueError:
        limit = 5  # default if user input is invalid

    # 2. Your Finnhub API key (replace with your actual key)
    FINNHUB_API_KEY = ""

    # 3. Retrieve news articles
    articles = get_finnhub_news(FINNHUB_API_KEY, ticker)

    # 4. Apply client-side limit
    articles = articles[:limit]

    # 5. Print out the results
    if not articles:
        print("No articles found or there was an error.")
        return

    print(f"\n=== Top {limit} News Articles for {ticker} (Last 30 Days) ===")
    for i, article in enumerate(articles, start=1):
        title = article.get("headline", "No Title")
        summary = article.get("summary", "No Summary")
        url = article.get("url", "No URL")
        source = article.get("source", "No Source")
        published_timestamp = article.get("datetime", 0)

        # Convert Unix timestamp to human-readable date
        published_date = datetime.utcfromtimestamp(published_timestamp).strftime("%Y-%m-%d %H:%M:%S")

        print(f"\nArticle {i}:")
        print(f"Title: {title}")
        print(f"Source: {source}")
        print(f"Published: {published_date}")
        print(f"Summary: {summary[:200]}...")  # limit summary preview
        print(f"Link: {url}")

if __name__ == "__main__":
    main()

Request URL: https://finnhub.io/api/v1/company-news?symbol=AAPL&from=2025-02-02&to=2025-03-04&token=cuu02a9r01qv6ijkqr0gcuu02a9r01qv6ijkqr10
Error fetching news from Finnhub: {'error': 'Invalid API key'}
No articles found or there was an error.


## Polgyon.io API

In [ ]:
def get_polygon_news(api_key, ticker, limit=5):
    """
    Fetches up to `limit` news articles about `ticker` from the Polygon.io "reference/news" endpoint.
    Returns a list of articles (each is a dict with title, author, description, etc.).
    """

    url = "https://api.polygon.io/v2/reference/news"
    params = {
        "ticker": ticker,
        "limit": limit,
        "apiKey": api_key
    }

    response = requests.get(url, params=params)
    print("Request URL:", response.url)  # For debugging, see the final request

    data = response.json()

    # Polygon.io returns articles under the "results" key
    # Example response structure:
    # {
    #   "status": "OK",
    #   "count": 5,
    #   "results": [
    #       { "title": "...", "article_url": "...", ... },
    #       ...
    #   ]
    # }
    if "results" not in data:
        print(f"Error fetching news from Polygon.io: {data}")
        return []

    return data["results"]


def main():
    # 1. Prompt user for inputs
    ticker = input("Enter the company ticker symbol (e.g., AAPL, TSLA): ").upper()
    limit_str = input("How many news articles would you like to see? (default 5): ")

    # 2. Convert limit input to int or default to 5
    try:
        limit = int(limit_str)
    except ValueError:
        limit = 5

    # 3. Your Polygon.io API key
    #    (Replace with your actual API key; do NOT hardcode in production)
    POLYGON_API_KEY = ""

    # 4. Retrieve news articles
    articles = get_polygon_news(POLYGON_API_KEY, ticker, limit=limit)

    # 5. Print out the results
    if not articles:
        print("No articles found or there was an error.")
        return

    print(f"\n=== Top {limit} News Articles for {ticker} ===")
    for i, article in enumerate(articles, start=1):
        title = article.get("title", "No Title")
        article_url = article.get("article_url", "No URL")
        author = article.get("author", "No Author")
        published_utc = article.get("published_utc", "No Date/Time")
        description = article.get("description", "No Description")

        print(f"\nArticle {i}:")
        print(f"Title: {title}")
        print(f"Author: {author}")
        print(f"Published (UTC): {published_utc}")
        print(f"Description: {description[:200]}...")  # Limit summary preview
        print(f"Link: {article_url}")

if __name__ == "__main__":
    main()

Request URL: https://api.polygon.io/v2/reference/news?ticker=AAPL&limit=5&apiKey=ymrniCK302SrpiQ3WIISSIpIUNbTYSYt

=== Top 5 News Articles for AAPL ===

Article 1:
Title: Palantir Extends Losses, Microsoft Slides: What's Driving Markets Monday?
Author: Erica Kollmann
Published (UTC): 2025-02-24T17:54:27Z
Description: The major indices struggled to maintain early momentum on Monday, with Microsoft, Broadcom, Tesla, and Meta Platforms facing pressure, while Apple announced plans for a $500 billion investment in the ...
Link: https://www.benzinga.com/general/market-summary/25/02/43920295/palantir-extends-losses-microsoft-slides-whats-driving-markets-monday

Article 2:
Title: Should You Buy Apple Stock Instead of Microsoft Stock?
Author: Parkev Tatevosian, Cfa
Published (UTC): 2025-02-24T14:36:11Z
Description: The article compares Apple and Microsoft as long-term investments, noting that only one can be the better choice. It provides analysis and recommendations from The Motley Fool....
Li

## Marketaux API

In [ ]:
def get_marketaux_news(api_token, symbols, limit=5):
    """
    Fetches up to `limit` news articles for the given `symbols` from the Marketaux 'v1/news/all' endpoint.
    `symbols` can be a single ticker like 'AAPL' or multiple tickers like 'TSLA,AMZN,MSFT'.
    
    Returns a list of article objects (dicts) with fields such as 'title', 'description', 'source', etc.
    """

    base_url = "https://api.marketaux.com/v1/news/all"
    
    # Build query parameters. We'll include `filter_entities` and `language` as shown in the example.
    # You can adjust or remove them as needed.
    params = {
        "symbols": symbols,           # e.g. "TSLA,AMZN,MSFT" or "AAPL"
        "filter_entities": "true",    # whether to extract entities from the text
        "language": "en",            # language filter for articles
        "api_token": api_token
        # Note: The Marketaux example doesn't show a built-in 'limit' param,
        # so we'll apply the limit client-side below.
    }

    response = requests.get(base_url, params=params)
    print("Request URL:", response.url)  # Debugging: see the final request
    data = response.json()

    # Marketaux's response typically has a structure like:
    # {
    #   "data": [ ... list of articles ... ],
    #   "meta": { ... pagination info ... }
    # }
    # Check if "data" exists in the response
    if "data" not in data:
        print(f"Error fetching news from Marketaux: {data}")
        return []

    articles = data["data"]
    
    # Since Marketaux doesn't necessarily limit results on the server side by default,
    # apply the limit client-side here
    return articles[:limit]


def main():
    # Prompt user for inputs
    symbols = input("Enter one or more comma-separated tickers (e.g., AAPL or TSLA,AMZN,MSFT): ").upper()
    limit_str = input("How many news articles would you like to see? (default 5): ")

    # Convert limit input to int or default to 5
    try:
        limit = int(limit_str)
    except ValueError:
        limit = 5

    # Replace with your actual Marketaux API key
    MARKETAUX_API_TOKEN = ""

    # Fetch the news articles
    articles = get_marketaux_news(MARKETAUX_API_TOKEN, symbols, limit=limit)

    if not articles:
        print("No articles found or there was an error.")
        return

    print(f"\n=== Top {limit} News Articles for {symbols} ===")
    for i, article in enumerate(articles, start=1):
        title = article.get("title", "No Title")
        description = article.get("description", "No Description")
        source = article.get("source", "No Source")
        published_at = article.get("published_at", "No Publish Time")
        url = article.get("url", "No URL")

        print(f"\nArticle {i}:")
        print(f"Title: {title}")
        print(f"Source: {source}")
        print(f"Published: {published_at}")
        print(f"Description: {description[:200]}...")  # truncate for preview
        print(f"Link: {url}")

if __name__ == "__main__":
    main()

Request URL: https://api.marketaux.com/v1/news/all?symbols=AAPL&filter_entities=true&language=en&api_token=yiVSqYtyxfk1PLFSgY7REStRB3GcnJUPLeLZFNJH

=== Top 3 News Articles for AAPL ===

Article 1:
Title: ASX Market Open: Dip buying Down Under won’t be enough to keep Week 9 from turning red again | Feb 25, 2025
Source: themarketherald.com.au
Published: 2025-02-24T21:03:28.000000Z
Description: After barely edging into the green late yesterday, Australian shares are now…...
Link: https://themarketonline.com.au/asx-market-open-dip-buying-down-under-wont-be-enough-to-keep-week-9-from-turning-red-again-feb-25-2025-2025-02-25/

Article 2:
Title: Inside Apple’s $500 Billion Investment
Source: medium.com
Published: 2025-02-24T20:48:33.000000Z
Description: "Apple commits $500 billion to U.S. innovation, manufacturing, and AI, marking a major shift in tech investment. Discover its impact on jobs, economy, and global tech."...
Link: https://medium.com/@vsinha027/inside-apples-500-billion-investme

# Exploring Alpha Vantage API

In [ ]:
import requests
import os

########################################
# Alpha Vantage: Various Endpoints
########################################

def get_alpha_vantage_news(api_key, ticker, topic=None, limit=5, sort_preference="RELEVANCE"):
    """
    Fetches news articles about `ticker` from the Alpha Vantage News API.
    Defaults to sort by RELEVANCE (unless overridden with sort_preference = "LATEST" or "EARLIEST").
    Optionally filters by `topic`, applies a limit client-side.

    Returns a list of dicts with article info (title, summary, sentiment, etc.).
    """

    url = "https://www.alphavantage.co/query"

    # Base params for the NEWS_SENTIMENT endpoint
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "apikey": api_key,
        # Default to RELEVANCE sorting
        "sort": sort_preference.upper()  # "RELEVANCE", "LATEST", or "EARLIEST"
    }

    if topic:
        params["topics"] = topic

    response = requests.get(url, params=params)
    print("Request URL:", response.url)  # Debugging: see the final request URL
    data = response.json()

    if "feed" not in data:
        print(f"Error fetching news from Alpha Vantage: {data}")
        return []

    articles = data["feed"]

    # Apply client-side limit
    return articles[:limit]


def get_company_overview(api_key, ticker):
    """
    Fetches the company overview (e.g., Name, Description, MarketCap, etc.)
    from Alpha Vantage using function=OVERVIEW.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "OVERVIEW",
        "symbol": ticker,
        "apikey": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()

    # If there's an error, data may contain something else
    if "Symbol" not in data:
        print(f"Error fetching company overview: {data}")
        return None

    return data


def get_etf_profile(api_key, ticker):
    """
    Fetches the ETF profile from Alpha Vantage using function=ETF_PROFILE.
    Includes data such as ETF holdings, net assets, etc. (Only works if ticker is an ETF).
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "ETF_PROFILE",
        "symbol": ticker,
        "apikey": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()

    # Check for basic structure
    if "symbol" not in data:
        print(f"Error fetching ETF profile: {data}")
        return None

    return data


def get_stock_price(api_key, ticker):
    """
    Fetches the TIME_SERIES_DAILY for the given ticker and returns the most recent daily data.
    You could also return the entire time series if desired.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",  # or TIME_SERIES_INTRADAY
        "symbol": ticker,
        "outputsize": "compact",         # or "full"
        "apikey": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()

    if "Time Series (Daily)" not in data:
        print(f"Error fetching stock price: {data}")
        return None

    # Extract the most recent date
    time_series = data["Time Series (Daily)"]
    most_recent_date = sorted(time_series.keys(), reverse=True)[0]
    most_recent_data = time_series[most_recent_date]
    return {
        "date": most_recent_date,
        "open": most_recent_data["1. open"],
        "high": most_recent_data["2. high"],
        "low":  most_recent_data["3. low"],
        "close":most_recent_data["4. close"],
        "volume":most_recent_data["5. volume"]
    }


########################################
# Main Interactive Script
########################################
def main():
    ALPHA_VANTAGE_API_KEY = ""

    # Prompt the user for a ticker
    ticker = input("Enter the company ticker symbol (e.g., AAPL, TSLA, SPY for ETF): ").upper()

    # Prompt user for the data type they want
    print("\nWhat type of data do you want?")
    print("1) News")
    print("2) Company Overview")
    print("3) ETF Profile")
    print("4) Stock Price (Daily)")
    choice = input("Enter 1/2/3/4: ").strip()

    if choice == "1":
        # NEWS
        limit_str = input("How many news articles would you like? (default 5): ")
        try:
            limit = int(limit_str)
        except ValueError:
            limit = 5

        # Prompt for optional topic
        print("\nAvailable topics include: blockchain, earnings, ipo, finance, technology, etc.")
        topic_input = input("Enter a topic from the list above (or leave blank for none): ").strip()

        # Prompt for sort preference; default to RELEVANCE if blank or invalid
        print("\nSort options: RELEVANCE, LATEST, EARLIEST")
        sort_input = input("How would you like to sort the news? (default RELEVANCE): ").strip().upper()
        valid_sorts = ["RELEVANCE", "LATEST", "EARLIEST"]
        if sort_input not in valid_sorts:
            sort_input = "RELEVANCE"

        articles = get_alpha_vantage_news(
            api_key=ALPHA_VANTAGE_API_KEY,
            ticker=ticker,
            topic=topic_input if topic_input else None,
            limit=limit,
            sort_preference=sort_input
        )

        if not articles:
            print("No articles found or there was an error.")
            return

        print(f"\n=== Top {limit} {sort_input.title()} News Articles for {ticker} ===")
        for i, article in enumerate(articles, start=1):
            title = article.get("title", "No Title")
            summary = article.get("summary", "No Summary")
            sentiment = article.get("overall_sentiment_label", "No Sentiment")
            url = article.get("url", "No URL")

            print(f"\nArticle {i}:")
            print(f"Title: {title}")
            print(f"Sentiment: {sentiment}")
            print(f"Summary: {summary[:200]}...")
            print(f"Link: {url}")

    elif choice == "2":
        # COMPANY OVERVIEW
        overview_data = get_company_overview(ALPHA_VANTAGE_API_KEY, ticker)
        if not overview_data:
            return
        print(f"\n=== Company Overview for {ticker} ===")
        for key, val in overview_data.items():
            print(f"{key}: {val}")

    elif choice == "3":
        # ETF PROFILE
        etf_data = get_etf_profile(ALPHA_VANTAGE_API_KEY, ticker)
        if not etf_data:
            return
        print(f"\n=== ETF Profile for {ticker} ===")
        for key, val in etf_data.items():
            print(f"{key}: {val}")

    elif choice == "4":
        # STOCK PRICE (TIME_SERIES_DAILY)
        stock_data = get_stock_price(ALPHA_VANTAGE_API_KEY, ticker)
        if not stock_data:
            return
        print(f"\n=== Latest Stock Price for {ticker} ===")
        print(f"Date:  {stock_data['date']}")
        print(f"Open:  {stock_data['open']}")
        print(f"High:  {stock_data['high']}")
        print(f"Low:   {stock_data['low']}")
        print(f"Close: {stock_data['close']}")
        print(f"Volume:{stock_data['volume']}")

    else:
        print("Invalid choice. Exiting.")


if __name__ == "__main__":
    main()


What type of data do you want?
1) News
2) Company Overview
3) ETF Profile
4) Stock Price (Daily)

Available topics include: blockchain, earnings, ipo, finance, technology, etc.

Sort options: RELEVANCE, LATEST, EARLIEST
Request URL: https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=RAL8AFRU7F2Y9ZPI&sort=RELEVANCE

=== Top 5 Relevance News Articles for AAPL ===

Article 1:
Title: This Solar Company's 5-Year Return Makes Bitcoin, Dogecoin, Tesla, Disney, Apple, Microsoft, Ford And Amazon Look Trivial
Sentiment: Neutral
Summary: Enphase Energy Inc ( NASDAQ: ENPH ) is known as a global leader in solar energy solutions, and over the past five years, delivered beaming returns for traders and investors. Since May 2017, Enphase En...
Link: https://www.benzinga.com/markets/cryptocurrency/22/05/27337504/this-solar-companys-5-year-return-makes-bitcoin-dogecoin-tesla-disney-apple-microsoft-ford

Article 2:
Title: US Warns China Against Helping Russia In Ukraine War
Sen

# API workflow

In [2]:
import requests
import json
import pandas as pd

def get_alpha_vantage_news(api_key, ticker, sort_preference="RELEVANCE"):
    """
    Fetches news articles about `ticker` from Alpha Vantage's NEWS_SENTIMENT endpoint.
    Returns a JSON string.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "apikey": api_key,
        "sort": sort_preference.upper()  # RELEVANCE, LATEST, EARLIEST
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "feed" not in data:
        print(f"Error fetching news from Alpha Vantage: {data}")
        return None

    return json.dumps(data, indent=4)  # Convert JSON to a readable string


def get_company_overview(api_key, ticker):
    """
    Fetches the company overview from Alpha Vantage using function=OVERVIEW.
    Returns a Pandas DataFrame.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "OVERVIEW",
        "symbol": ticker,
        "apikey": api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "Symbol" not in data:
        print(f"Error fetching company overview: {data}")
        return None

    # Convert JSON dictionary to a Pandas DataFrame
    overview_df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])
    overview_df.reset_index(inplace=True)
    overview_df.columns = ['Field', 'Value']

    return overview_df


def main():
    ALPHA_VANTAGE_API_KEY = "YOUR_ALPHA_VANTAGE_API_KEY"

    # Prompt user for a ticker symbol
    ticker = input("Enter the company ticker symbol (e.g., AAPL, TSLA): ").upper()

    print("\nFetching company overview and news articles...\n")

    # Fetch data
    company_overview_df = get_company_overview(ALPHA_VANTAGE_API_KEY, ticker)
    news_json = get_alpha_vantage_news(ALPHA_VANTAGE_API_KEY, ticker)

    if company_overview_df is None and not news_json:
        print("No data found or there was an error.")
        return

    # Print the Company Overview DataFrame
    if company_overview_df is not None:
        print("\n=== Company Overview ===")
        print(company_overview_df)

    # Print the News Articles JSON
    if news_json:
        print("\n=== News Articles ===\n")
        print(news_json)

    return company_overview_df, news_json


if __name__ == "__main__":
    company_overview_df, final_news_text = main()



Fetching company overview and news articles...


=== Company Overview ===
                         Field  \
0                       Symbol   
1                    AssetType   
2                         Name   
3                  Description   
4                          CIK   
5                     Exchange   
6                     Currency   
7                      Country   
8                       Sector   
9                     Industry   
10                     Address   
11                OfficialSite   
12               FiscalYearEnd   
13               LatestQuarter   
14        MarketCapitalization   
15                      EBITDA   
16                     PERatio   
17                    PEGRatio   
18                   BookValue   
19            DividendPerShare   
20               DividendYield   
21                         EPS   
22          RevenuePerShareTTM   
23                ProfitMargin   
24          OperatingMarginTTM   
25           ReturnOnAssetsTTM   
26     